In [1]:
#declare dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests
import time
from selenium import webdriver

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [5]:
#find newest article title
article = soup.find('div', class_='list_text')
news_title = article.find('div', class_='content_title').text
print(news_title)

NASA's Curiosity Mars Rover Takes a New Selfie Before Record Climb


In [6]:
#newest article text
news_p = article.find('div', class_='article_teaser_body').text
print(news_p)

Along with capturing an image before its steepest ascent ever, the robotic explorer filmed its "selfie stick," or robotic arm, in action.


### Space Images

In [7]:
#URL of page to be scraped
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)

In [8]:
#full image
image_full=browser.find_by_id('full_image')
image_full.click()
time.sleep(2)
browser.click_link_by_partial_text('more info')

/Users/sarahparkington/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [9]:
image_html = browser.html

image_soup = bs(image_html, 'html.parser')

In [10]:
# Scrape the URL
img_url=image_soup.find('img', class_ = 'main_image')['src']
featured_img_url = "https://www.jpl.nasa.gov" + img_url
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19639_hires.jpg'

### Mars Facts

In [11]:
# Visit Mars Facts webpage for interesting facts about Mars
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)
html = browser.html

In [12]:
# Use Pandas to scrape the table containing facts about Mars
table = pd.read_html(facts_url)
mars_facts = table[1]

In [13]:
mars_facts

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-153 to 20 °C,-88 to 58°C


In [14]:
# Use Pandas to convert the data to a HTML table string
mars_facts.to_html('table.html')

### Mars Hemispheres

In [15]:
# Visit USGS webpage for Mars hemispehere images
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, "html.parser")

# Create dictionary to store titles & links to images
hemisphere_image_urls = []

# Retrieve all elements that contain image information
results = soup.find("div", class_ = "result-list" )
hemispheres = results.find_all("div", class_="item")

# Iterate through each image
for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})

# Print image title and url
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
